In [1]:
### imports


import pandas as pd # DataFrame Manipulation Package
import numpy as np
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer # Convert a collection of raw documents to a matrix of TF-IDF features
from sklearn.feature_extraction.text import CountVectorizer # Convert a collection of text documents to a matrix of token counts
from sklearn.decomposition import LatentDirichletAllocation # Latent Dirichlet Allocation is a topic model that is used for discovering abstract topics from a collection of documents (variational Bayes algorithm)

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB # The multinomial Naive Bayes classifier is suitable for classification with discrete features (e.g., word counts for text classification)

import string # Collection of string operations
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer #Lemmatize using WordNet's built-in morphy function. Returns the input word unchanged if it cannot be found in WordNet.
from nltk import word_tokenize

from nltk.sentiment.util import mark_negation


In [2]:
### import stopwords

import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/jack/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jack/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jack/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
#### define preprocessing functions

def to_list(x):
    list_words = x.split(' ')
    return list_words

def to_string(x):
    string = " ".join(x)
    return string

#===============================================================

def punctuation(x):
    for punctuation in string.punctuation:
        x =  x.replace(punctuation, '')
    return x.lower()

def remove_numbers (x):
    words_only = ''.join([i for i in x if not i.isdigit()])
    return words_only

def m_negation(x):
    tokenized = word_tokenize(x)
    without_stopwords = [word for word in tokenized if not word in stop_words]
    tokenized_neg = mark_negation(without_stopwords)
    return tokenized_neg

def remove_stopwords(x):
    tokenized = word_tokenize(x)
    without_stopwords = [word for word in tokenized if not word in stop_words]
    return without_stopwords

def remove_our_stopwords(x):
    tokenized = word_tokenize(x)
    without_stopwords = [word for word in tokenized if not word in OUR_STOPWORDS]
    return without_stopwords

def lemmatize_review(x):
    lemma = WordNetLemmatizer()
    lista = []
    for w in x:
       lista.append(lemma.lemmatize(w))
    return lista

#===============================================================

def count_words(x):
    wordfreq = []
    for w in x:
        wordfreq.append(x.count(w))
    return dict(zip(x, wordfreq))

def total_count(x):
    total_count = {}
    for row in x:
        for key in row.keys():
          if key in total_count:
              total_count[key] += 1
          else:
              total_count[key] = 1
    return pd.DataFrame(sorted(total_count.items(), key=lambda x: x[1], reverse=True)).head(30).T

def round_two(x):
    return str(int(round(x, 2) * 100)) + "%"

def one_or_zero(x):
    # Makes the prediction a binary outpur
    if x > 0.5:
        x = 1
    else:
        x = 0
    return x

#===============================================================

def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], round(topic[i], 2))
                        for i in topic.argsort()[:-10 - 1:-1]])

In [4]:
side_effects = pd.read_csv('../raw_data/frequent_adr.csv')

side_effects.head(3)

,0
0,Abdominal pain
1,Gastrointestinal pain
2,Amblyopia


In [7]:
### read list into DF and preprocess side effects

stop_words = set(stopwords.words('english')) 

side_effects = pd.read_csv('../raw_data/frequent_adr.csv')

side_effects["Side_Effect"] = pd.DataFrame(side_effects)

side_effects["Side_Effect"] = side_effects.Side_Effect.apply(punctuation)
side_effects["Side_Effect"] = side_effects.Side_Effect.apply(remove_numbers)
side_effects["Side_Effect"] = side_effects.Side_Effect.apply(to_list)
side_effects["Side_Effect"] = side_effects.Side_Effect.apply(lemmatize_review)
side_effects["Side_Effect"] = side_effects.Side_Effect.apply(to_string)
side_effects["Side_Effect"] = side_effects.Side_Effect.apply(remove_stopwords)
side_effects = side_effects["Side_Effect"].tolist()

SE_LIST = [] 

for sublist in side_effects: 
    for item in sublist:
        if item not in SE_LIST:
            SE_LIST.append(item)

# SE_LIST

In [8]:
OUR_STOPWORDS = set(stopwords.words('english'))

for effect in SE_LIST: 
    OUR_STOPWORDS.add(effect)

for word in stop_words:
    OUR_STOPWORDS.remove(word)

In [9]:
### manually define side effect stop words

OUR_STOPWORDS = ['abdominal', 'constipation', 'diarrhea', 'skin', 'rash', 'vertigo', 'dizziness', 'drowsiness',
              'headache', 'mood disorders', 'insomnia', 'mood swings', "no side effect", "good response", "improvement"]

In [13]:
### test side effect selection function

list_merge_SE = ['juan', 'jack', 'peter', 'hendrike', 'headache']

def side_effects_lst(x):
    lista = []
    for i in x:
        y = remove_our_stopwords(i)
        if i not in y:
            lista.append(i)
    return lista

side_effects_lst(list_merge_SE)

['headache']

In [25]:
### read in all data

data = pd.read_csv('../raw_data/drugsComTrain_raw.csv')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161297 entries, 0 to 161296
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   uniqueID     161297 non-null  int64 
 1   drugName     161297 non-null  object
 2   condition    160398 non-null  object
 3   review       161297 non-null  object
 4   rating       161297 non-null  int64 
 5   date         161297 non-null  object
 6   usefulCount  161297 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 8.6+ MB


In [60]:
#data = pd.read_csv('drugsComTrain_raw.csv')



data["clean_review"] = data["review"].apply(punctuation)
data['clean_review'] = data.clean_review.apply(remove_numbers)
data['clean_review_lst'] = data.clean_review.apply(to_list)

data["NonStopwords_review_lst"] = data.clean_review.apply(remove_stopwords)
data["NonStopwords_review_str"] = data.NonStopwords_review_lst.apply(to_string)

data["NonStopwords_review_lst_MN"] = data.clean_review.apply(m_negation)
data["NonStopwords_review_str_MN"] = data.NonStopwords_review_lst_MN.apply(to_string)

data["Lemmatized_review_lst"] = data.NonStopwords_review_lst_MN.apply(lemmatize_review)
data["Lemmatized_review_str"] = data.Lemmatized_review_lst.apply(to_string)

data["Lemmatized_review_list"] = data.NonStopwords_review_lst.apply(lemmatize_review)
data["Lemmatized_review"] = data.Lemmatized_review_list.apply(to_string)
data["Lemmatized_review_list"] = data.Lemmatized_review.apply(remove_stopwords)
data["Lemmatized_review"] = data.Lemmatized_review_list.apply(to_string)
# data["our"] = data.Lemmatized_review.apply(remove_our_stopwords)
# data["our_str"] = data.our.apply(to_string)

data["words_count"] = data.Lemmatized_review_list.apply(count_words)

data = data.drop(["clean_review", "clean_review_lst", "NonStopwords_review_lst", "date", "NonStopwords_review_str"], axis = 1)

data["Side_Effects_mention"] = data.Lemmatized_review_list.apply(side_effects_lst)

data.head(10)

KeyError: "['date'] not found in axis"

In [58]:
data.head(100)

,uniqueID,drugName,condition,review,rating,usefulCount,NonStopwords_review_lst_MN,NonStopwords_review_str_MN,Lemmatized_review_lst,Lemmatized_review_str,Lemmatized_review_list,Lemmatized_review,words_count,Side_Effects_mention
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,27,"[side, effect, take, combination, bystolic, mg...",side effect take combination bystolic mg fish oil,"[side, effect, take, combination, bystolic, mg...",side effect take combination bystolic mg fish oil,"[side, effect, take, combination, bystolic, mg...",side effect take combination bystolic mg fish oil,"{'side': 1, 'effect': 1, 'take': 1, 'combinati...",constipation
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,192,"[son, halfway, fourth, week, intuniv, became, ...",son halfway fourth week intuniv became concern...,"[son, halfway, fourth, week, intuniv, became, ...",son halfway fourth week intuniv became concern...,"[son, halfway, fourth, week, intuniv, became, ...",son halfway fourth week intuniv became concern...,"{'son': 1, 'halfway': 1, 'fourth': 1, 'week': ...",headache
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,17,"[used, take, another, oral, contraceptive, pil...",used take another oral contraceptive pill cycl...,"[used, take, another, oral, contraceptive, pil...",used take another oral contraceptive pill cycl...,"[used, take, another, oral, contraceptive, pil...",used take another oral contraceptive pill cycl...,"{'used': 1, 'take': 1, 'another': 1, 'oral': 1...",headache
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,10,"[first, time, using, form, birth, control, im,...",first time using form birth control im glad we...,"[first, time, using, form, birth, control, im,...",first time using form birth control im glad we...,"[first, time, using, form, birth, control, im,...",first time using form birth control im glad we...,"{'first': 3, 'time': 1, 'using': 2, 'form': 1,...",improvement
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,37,"[suboxone, completely, turned, life, around, f...",suboxone completely turned life around feel he...,"[suboxone, completely, turned, life, around, f...",suboxone completely turned life around feel he...,"[suboxone, completely, turned, life, around, f...",suboxone completely turned life around feel he...,"{'suboxone': 3, 'completely': 1, 'turned': 1, ...",improvement
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,45237,Fluoxetine,Major Depressive Disorde,"""I started Prozac as one of my first anti depr...",2,18,"[started, prozac, one, first, anti, depressant...",started prozac one first anti depressants mdd ...,"[started, prozac, one, first, anti, depressant...",started prozac one first anti depressant mdd m...,"[started, prozac, one, first, anti, depressant...",started prozac one first anti depressant mdd m...,"{'started': 2, 'prozac': 1, 'one': 2, 'first':...",improvement
96,102810,Aripiprazole,Depression,"""Intake Effexor XR 375 mg, and lorazepam for d...",4,33,"[intake, effexor, xr, mg, lorazepam, depressio...",intake effexor xr mg lorazepam depression anxi...,"[intake, effexor, xr, mg, lorazepam, depressio...",intake effexor xr mg lorazepam depression anxi...,"[intake, effexor, xr, mg, lorazepam, depressio...",intake effexor xr mg lorazepam depression anxi...,"{'intake': 1, 'effexor': 1, 'xr': 1, 'mg': 1, ...",skin
97,60280,NuvaRing,Birth Control,"""I am torn by the Nuvaring. The convenience is...",5,0,"[torn, nuvaring, convenience, great, daily, ti...",torn nuvaring convenience great daily time tak...,"[torn, nuvaring, convenience, great, daily, ti...",torn nuvaring convenience great daily time tak...,"[torn, nuvaring, convenience, great, daily, ti...",torn nuvaring convenience great daily time tak...,"{'torn': 1, 'nuvaring': 3, 'convenience': 1, '...",headache
98,1067

In [61]:
countSE_drug_df = data[['drugName', 'Side_Effects_mention']]
countSE_drug_df.groupby(by='drugName')['Side_Effects_mention'].value_counts().unstack()


Side_Effects_mention,abdominal,constipation,diarrhea,dizziness,drowsiness,headache,improvement,insomnia,rash,skin,vertigo
drugName,,,,,,,,,,,
A + D Cracked Skin Relief,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A / B Otic,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
Abacavir / dolutegravir / lamivudine,NaN,NaN,3.0,1.0,NaN,3.0,1.0,3.0,1.0,3.0,NaN
Abacavir / lamivudine / zidovudine,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
Abatacept,NaN,1.0,NaN,1.0,NaN,2.0,2.0,NaN,NaN,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
Zyprexa Zydis,NaN,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN
Zyrtec,2.0,1.0,3.0,3.0,NaN,4.0,NaN,1.0,1.0,4.0,1.0
Zyvox,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,3.0,NaN


In [46]:
countSE_drug_df = data[['drugName', 'Side_Effects_mention']]
#countSE_drug_df = countSE_drug_df.reset_index()
countSE_drug_df.groupby(by='drugName')
#SE_count_drug.to_csv('SE_count_per_drug.csv')
countSE_drug_df.head(50)

,drugName,Side_Effects_mention
0,Valsartan,constipation
1,Guanfacine,headache
2,Lybrel,headache
3,Ortho Evra,improvement
4,Buprenorphine / naloxone,improvement
5,Cialis,diarrhea
6,Levonorgestrel,constipation
7,Aripiprazole,insomnia
8,Keppra,insomnia
9,Ethinyl estradiol / levonorgestrel,insomnia


In [30]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

X = vectorizer.fit_transform(data['Side_Effects_mention'])

X.toarray()

AttributeError: 'list' object has no attribute 'lower'